In [1]:
import pandas as pd


In [2]:
list_of_census = [
    "EW_1851",
    "EW_1861",
    "EW_1881",
    "EW_1891",
    "EW_1901",
    "EW_1911",
    "scot_1851",
    "scot_1861",
    "scot_1871",
    "scot_1881",
    "scot_1891",
    "scot_1901",
]

list_of_census = ["EW_1861"]

In [3]:
census_csv_params = {
	"sep": "\t",
	"encoding": "latin-1",
	"quoting": 3,
	"na_values": [".", " "],
	# "usecols": ["sha256"], #add other columns from census here
	# "nrows":100000
}

census_csv_params_2 = {
	"sep": "\t",
	"encoding": "latin-1",
	"quoting": 3,
	"na_values": [".", " "],
	"usecols": ["RecID"], #add other columns from census here
	# "nrows":100000
}


for census in list_of_census:
	country = census.split("_")[0]
	year = census.split("_")[1]
	anonymised_census = pd.read_csv(f"input/census/{country}{year}_geocode_egress_2022_11_03.txt", **census_csv_params)

	if country == "EW":
		if year in ["1851", "1861"]:
			suffix = "_v1"
		else:
			suffix = ""
	else:
		suffix = ""
	full_census = pd.read_csv(f"/Users/jrhodes/lwm_projects/Living-with-Machines-code/census/ICEM/{country}{str(year)[-2:]}outputfile{suffix}.txt",**census_csv_params_2)
	if country == "EW" and year in ["1851","1881"]:
		print(census)
		full_census = full_census.sort_values(by = ["RecID"]).reset_index()
	else:
		'do nothing'
	merged = pd.merge(left = anonymised_census, right = full_census, left_index=True, right_index=True,validate="one_to_one")
	merged.to_csv(f"input/census/{country}{year}_geocode_egress_2022_11_03_recid.txt", sep="\t", encoding="latin-1", quoting=3, na_rep = ".", index = False)
	merged

/usr/local/anaconda3/envs/census-place-linking/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,7,11,12,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
params = {
	"sep": "\t",
	"encoding": "latin-1",
	"quoting": 3,
	"na_values": [".", " "],
	"usecols": ["sha256","RecID"], #add other columns from census here
	# "nrows":100000
}

census = pd.read_csv("input/census/EW1881_geocode_egress_2022_11_03_recid.txt",**params)
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26124585 entries, 0 to 26124584
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   sha256  object
 1   RecID   int64 
dtypes: int64(1), object(1)
memory usage: 398.6+ MB


In [12]:
census["n"] = census["sha256"].str.extract('(\d)')
census["n"] = census["n"].astype(int)
census["RecID"] = census["RecID"].astype(int)
census["check"] = census["RecID"] * census["n"]
census["check"].mean()

58772826.38370041

In [10]:
census

,sha256,RecID,n,check
0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,1,6,6
1,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,4,8
2,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,3,4,12
3,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,4,4,16
4,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,5,2,10
...,...,...,...,...
17711039,72adcc019f338fcbc6523a583b248273af4ba9fe36d81d...,17711040,7,123977280
17711040,ae45db4c751a312e09ea8b6323f35bb9ae1739a6c203d2...,17711041,4,70844164
17711041,c60f78250f80573572b533eff0a3810edadadb0000fb88...,17711042,6,106266252
17711042,1eb19955233a9d5f8d8053a7ee8ba53927f4e99f6952de...,17711043,1,17711043


In [13]:
census["RecID_offset"] = census["RecID"] - 2

In [14]:
import numpy as np
census["match"] = np.where(census.index == census["RecID_offset"],"MATCH","NO_MATCH")

In [15]:
census[census["match"] == "NO_MATCH"]

,sha256,RecID,n,check,RecID_offset,match


In [11]:
census = census.sort_values(by=["RecID"])

In [50]:
census

,sha256,RecID,n,check,RecID_offset,match
0,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,4,4,16,0,MATCH
1,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,5,2,10,1,MATCH
2,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,6,7,42,2,MATCH
3,7902699be42c8a8e46fbbb4501726517e86b22c56a189f...,7,7,49,3,MATCH
4,2c624232cdd221771294dfbb310aca000a0df6ac8b66b6...,8,2,16,4,MATCH
...,...,...,...,...,...,...
26124580,f415b995f61579872a906e06104bab6bea901f8dad3479...,26124584,4,104498336,26124580,MATCH
26124561,c67887969061bac00926db87a3448eeb8163b84f9a77c1...,26124585,6,156747510,26124581,NO_MATCH
26124567,151c1a34c3e299dc81d8df4b225e873b0c5b276454fc71...,26124586,1,26124586,26124582,NO_MATCH
26124571,7f478622ca7577013ed358aa4e841a1a16a38f8c10c2d2...,26124587,7,182872109,26124583,NO_MATCH
